# 100k DOFs case

From the Kratos outputs it seems that on every "Fluid Dynamics Analysis" step 6
problems are solved using AMGCL: 2 larger ones, and 4 smaller ones (a larger
problem has exactly 4 times more DOFs than a smaller one). Riccardo says these
are all physically different problems, which seems to be confirmed by the fact
that each of the problems takes distinctive and repeated across time steps
number of iterations to converge. I'll consider each of these as a separate
sequence of problems.

AMGCL parameters:

- `solver.type=bicgstab`
- `solver.tol=1e-6`
- `precond.coarse_enough=500`
- `precond.coarsening.type=aggregation`
- `precond.relax.type=damped_jacobi`

The larger problems were solved with 4x4 block values both with OpenMP and VexCL(CUDA) backends.

In [1]:
import pandas
import re
from pathlib import Path
def csv(fname):
    return pandas.read_csv(fname, delimiter='\t', header=None)

def speedup(full, part):
    data = {}
    for fname in (full, part):
        for line in open(fname):
            if m := re.search(r'amgcl:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'total')] = float(m.group(1))
            elif m := re.search(r'setup:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'setup')] = float(m.group(1))
    print(f"Total speedup: {100 * (data.get((full, 'total'), 0) / data.get((part, 'total'), 1) - 1):.2f}%")
    print(f"Setup speedup: {100 * (data.get((full, 'setup'), 0) / data.get((part, 'setup'), 1) - 1):.2f}%")
    
def report(size, device, num):
    full = Path(f'{size}/{device}-{num}-full.time')
    part = Path(f'{size}/{device}-{num}-part.time')
    
    print('*** Full AMGCL rebuild on every step ***\n')
    print(full.read_text().strip())
    print(f"\nAverage iterations: {csv(full.with_suffix('.log')).mean()[2]:.1f}")
    
    print('\n\n*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***\n')
    print(part.read_text().strip())
    print(f"\nAverage iterations: {csv(part.with_suffix('.log')).mean()[2]:.1f}")
    speedup(full, part)

## Small problem 0, OpenMP

In [2]:
report('small', 'cpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:        4.573 s] (100.00%)
[ self:          0.015 s] (  0.32%)
[  amgcl:        4.292 s] ( 93.85%)
[    setup:      1.462 s] ( 31.97%)
[      full:     1.462 s] ( 31.97%)
[    solve:      2.830 s] ( 61.87%)
[  reading:      0.267 s] (  5.83%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           4.275 s] (100.00%)
[ self:             0.010 s] (  0.23%)
[  amgcl:           3.927 s] ( 91.86%)
[    setup:         0.868 s] ( 20.29%)
[      full:        0.137 s] (  3.21%)
[      partial:     0.730 s] ( 17.08%)
[    solve:         3.060 s] ( 71.57%)
[  reading:         0.338 s] (  7.91%)

Average iterations: 5.0
Total speedup: 9.29%
Setup speedup: 68.43%


## Small problem 0, VexCL(CUDA)

In [3]:
report('small', 'gpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:        3.076 s] (100.00%)
[ self:          0.008 s] (  0.27%)
[  amgcl:        2.801 s] ( 91.06%)
[    setup:      2.015 s] ( 65.51%)
[      full:     2.015 s] ( 65.51%)
[    solve:      0.786 s] ( 25.55%)
[  reading:      0.267 s] (  8.67%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           2.668 s] (100.00%)
[ self:             0.008 s] (  0.32%)
[  amgcl:           2.376 s] ( 89.03%)
[    setup:         1.583 s] ( 59.31%)
[      full:        0.212 s] (  7.95%)
[      partial:     1.370 s] ( 51.36%)
[    solve:         0.793 s] ( 29.72%)
[  reading:         0.284 s] ( 10.65%)

Average iterations: 5.0
Total speedup: 17.89%
Setup speedup: 27.29%


## Small problem 2, OpenMP

In [4]:
report('small', 'cpu', 2)

*** Full AMGCL rebuild on every step ***

[Profile:       11.472 s] (100.00%)
[ self:          0.015 s] (  0.13%)
[  amgcl:       11.195 s] ( 97.58%)
[    setup:      1.600 s] ( 13.94%)
[      full:     1.600 s] ( 13.94%)
[    solve:      9.595 s] ( 83.64%)
[  reading:      0.263 s] (  2.29%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:          11.050 s] (100.00%)
[  amgcl:          10.776 s] ( 97.52%)
[    setup:         0.951 s] (  8.60%)
[      full:        0.155 s] (  1.40%)
[      partial:     0.796 s] (  7.20%)
[    solve:         9.825 s] ( 88.92%)
[  reading:         0.263 s] (  2.38%)

Average iterations: 15.6
Total speedup: 3.89%
Setup speedup: 68.24%


## Small problem 2, VexCL(CUDA)

In [5]:
report('small', 'gpu', 2)

*** Full AMGCL rebuild on every step ***

[Profile:        4.973 s] (100.00%)
[ self:          0.008 s] (  0.16%)
[  amgcl:        4.700 s] ( 94.51%)
[    setup:      2.047 s] ( 41.15%)
[      full:     2.047 s] ( 41.15%)
[    solve:      2.654 s] ( 53.35%)
[  reading:      0.265 s] (  5.33%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           4.471 s] (100.00%)
[ self:             0.008 s] (  0.18%)
[  amgcl:           4.199 s] ( 93.92%)
[    setup:         1.560 s] ( 34.89%)
[      full:        0.211 s] (  4.71%)
[      partial:     1.349 s] ( 30.18%)
[    solve:         2.639 s] ( 59.02%)
[  reading:         0.264 s] (  5.91%)

Average iterations: 15.6
Total speedup: 11.93%
Setup speedup: 31.22%


## Large problem 0, OpenMP

In [6]:
report('large', 'cpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:      363.478 s] (100.00%)
[  amgcl:      122.196 s] ( 33.62%)
[    setup:      7.634 s] (  2.10%)
[      full:     7.634 s] (  2.10%)
[    solve:    114.562 s] ( 31.52%)
[  reading:    241.226 s] ( 66.37%)

Average iterations: 26.3


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         358.541 s] (100.00%)
[  amgcl:         121.628 s] ( 33.92%)
[    setup:         6.363 s] (  1.77%)
[      full:        0.782 s] (  0.22%)
[      partial:     5.581 s] (  1.56%)
[    solve:       115.265 s] ( 32.15%)
[  reading:       236.859 s] ( 66.06%)

Average iterations: 26.4
Total speedup: 0.47%
Setup speedup: 19.97%


## Large problem 0, VexCL(CUDA)

In [7]:
report('large', 'gpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:      285.415 s] (100.00%)
[  amgcl:       46.599 s] ( 16.33%)
[    setup:     13.348 s] (  4.68%)
[      full:    13.348 s] (  4.68%)
[    solve:     33.250 s] ( 11.65%)
[  reading:    238.737 s] ( 83.65%)

Average iterations: 26.4


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         285.701 s] (100.00%)
[  amgcl:          44.765 s] ( 15.67%)
[    setup:        11.312 s] (  3.96%)
[      full:        1.540 s] (  0.54%)
[      partial:     9.772 s] (  3.42%)
[    solve:        33.453 s] ( 11.71%)
[  reading:       240.843 s] ( 84.30%)

Average iterations: 26.5
Total speedup: 4.10%
Setup speedup: 18.00%
